# **Analysis of Heart Disease Mortality Data**

**Samarth Tuli, Ndenko Fontem, Yi Zhu, Kimia Samieinejad**    


**Importance of Heart Disease Mortality Research**    

**What is coronary heart disease?**

**Tutorial Purpose**

In [5]:
print("Hello World. Testing notebook appears on GitHub page.")

Hello World. Testing notebook appears on GitHub page.


In [6]:
print("Yi Zhu testing")

Yi Zhu testing


In [ ]:
print("Ndenko testing")